In [7]:
import cv2
import numpy as np

cap = cv2.VideoCapture('videos/face.mp4')

while True:
    ret, frame = cap.read()
    face_casc = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    face_rects = face_casc.detectMultiScale(frame)
    if len(face_rects)>0:
        print(face_rects)
        break
# print(face_rects)

# converting list into tuple
face_x, face_y, w, h = tuple(face_rects[0])
track_window = (face_x, face_y, w, h)

# ROI for tracking
roi = frame[face_y:face_y+h,
           face_x:face_x+w]

# hsv color mapping
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)


[[105  67 205 205]]


In [8]:
# histogram to target on each frame for mean shift calculation
roi_hist = cv2.calcHist([hsv_roi],
                       [0],
                       None,
                       [180],
                       [0,180])

# normalizing the histogram
cv2.normalize(roi_hist,
             roi_hist,
             0,
             255,
             cv2.NORM_MINMAX)

# setting termination criteria
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)


In [9]:
while True:
    ret, frame = cap.read()
    
    if ret==True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        dest = cv2.calcBackProject([hsv],
                                      [0],
                                      roi_hist,
                                      [0,180],
                                      1)
        # Cam shift to get new coordinated of rectangle
        ret, track_window = cv2.CamShift(dest,
                                         track_window,
                                         term_crit)
        # drawing new rectangle
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame,
                            [pts],
                            True,
                            (0,255,0),
                            5)
        cv2.imshow("Cam Shift", img2)
        
        if cv2.waitKey(30) & 0xFF==27:
            break
    else:
        break
        
cap.release()
cv2.destroyAllWindows()